In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import urllib
import time
import csv
import os

In [1]:
place=["台北一"]
#crop_num=["SA1","SA2","SA3","SA4","SA5","SA6","SA0","SA9","SB1","SB2","SB9","SC1","SC9","SD1","SD9","SI1","SJ1","SJ2","SJ3","SJ4","SJ9","SL1","SO1","SO2","SO3","SO0","SP1","SP2","SP3","SP0","SP9","SV1","SV2","SV3","SV9"]
#crop_name=["蘿蔔-梅花","蘿蔔-椿仔","蘿蔔-矸仔","蘿蔔-櫻桃","蘿蔔-青色","蘿蔔-白玉","蘿蔔-其他","蘿蔔-進口","胡蘿蔔-未洗","胡蘿蔔-清洗","胡蘿蔔-進口","馬鈴薯-本產","馬鈴薯-進口","洋蔥-本產","洋蔥-進口","萵苣莖","芋-麵芋","芋-檳榔心芋","芋-里芋","芋-芋莖","芋-進口","豆薯","甘藷-臺農57號","甘藷-芋仔甘藷","甘藷-臺農66號","甘藷-其他","薑-老薑","薑-嫩薑","薑-粉薑","薑-其他","薑-進口","蘆筍-白蘆筍","蘆筍-綠蘆筍","蘆筍-蘆筍花","蘆筍-進口"]
crop_num=["FB12"]
crop_name=["花椰菜-白梗"]
col_name=["date","crop_num","crop_name","market_num","market_name","high","medium","low","mean","volume"] 
for a in place:
    for b in range(0,len(crop_name)):
        df=pd.DataFrame(columns=col_name)
        skip_num=0
        temp_str=""
        for j in range(0,1000): 
            url="https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?$top=1000&$skip="+str(skip_num)+"&Crop="+urllib.parse.quote(crop_name[b])+"&StartDate=101.01.01&EndDate=108.05.06&Market="+urllib.parse.quote(a) 
            get = json.loads(urlopen(url).read().decode('utf-8')) 
            data = [] 
            for i in range(0,len(get)):
                temp_str=get[0]['交易日期']
                Date=get[i]['交易日期'].split('.')
                year,month,date=Date[0],Date[1],Date[2]
                year=str(int(year)+1911)
                get[i]['交易日期']=year+'/'+month+'/'+date
                if get[i]['作物名稱']==crop_name[b]:
                    data.append({"date":get[i]['交易日期'],"crop_num":get[i]['作物代號'],"crop_name":get[i]['作物名稱'],"market_num":get[i]['市場代號'],"market_name":get[i]['市場名稱'],"high":get[i]['上價'],"medium":get[i]['中價'],"low":get[i]['下價'],"mean":get[i]['平均價'],"volume":get[i]['交易量']}) 
                #else:
                    #data.append({"date":get[i]['交易日期'],"crop_num":get[i]['作物代號'],"crop_name":get[i]['作物名稱'],"market_num":get[i]['市場代號'],"market_name":get[i]['市場名稱'],"high":0,"medium":0,"low":0,"mean":0,"volume":0}) 
            data.reverse() 
            df=pd.concat([pd.DataFrame(data), df], ignore_index=True,sort=True)
            if len(get)<1000: 
                break 
            skip_num=skip_num+1000 
            time.sleep(2000.0/1000.0)
        path=r"C:\Users\admin\Desktop\Project\test\test\\"+crop_name[b]+".csv"
        df["date"] = pd.to_datetime(df["date"])
        #df=df.drop(["date"], axis=1)
        df = df.drop_duplicates(subset=['date'], keep='first')
        df.to_csv(path, encoding='utf_8_sig')
        time.sleep(3000.0/1000.0)

NameError: name 'pd' is not defined

In [3]:
place_name=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北"]
place_num=["C0K240","467480","C0G620","467410","467440","467590","467490","C0E420","C0C480","466920"]
date=[]
for year in ['2012','2013','2014','2015','2016','2017','2018']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        date.append('-'.join([year,month]))
date.append('2019-01')
date.append('2019-02')
date.append('2019-03')
for a in range(0,len(place_name)):
    flag=0
    name=urllib.parse.quote(urllib.parse.quote(place_name[a]))
    for dd in date:
        url="http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain"+"&station="+place_num[a]+"&stname="+name+"&datepicker="+str(dd)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        trs = soup.findAll('tr')
        ths = trs[2].findAll('th')
        day=1
        data=[]
        fdata=[]
        if flag==0:
            title=[]
            for th in ths:
                title.append(th.text)
            title.pop(0)
            title.append("year")
            title.append("month")
            title.append("day")
            title.append("date")
            df=pd.DataFrame(columns=title)
            flag=1
        for tr in trs[4:]:
            t=tr.findAll('td')
            for x in t:
                data.append(x.text.strip())
            sp=dd.split('-')
            data.pop(0)
            data.append(sp[0])
            data.append(sp[1])
            data.append(str(day))
            data.append(sp[0]+'/'+sp[1]+'/'+str(day))
            fdata.append({"測站氣壓(hPa)":data[0],"海平面氣壓(hPa)":data[1],"測站最高氣壓(hPa)":data[2],"測站最高氣壓時間(LST)":data[3],"測站最低氣壓(hPa)":data[4],"測站最低氣壓時間(LST)":data[5],"氣溫(℃)":data[6],"最高氣溫(℃)":data[7],"最高氣溫時間(LST)":data[8],"最低氣溫(℃)":data[9],"最低氣溫時間(LST)":data[10],"露點溫度(℃)":data[11],"相對溼度(%)":data[12],"最小相對溼度(%)":data[13],"最小相對溼度時間(LST)":data[14],"風速(m/s)":data[15],"風向(360degree)":data[16],"最大陣風(m/s)":data[17],"最大陣風風向(360degree)":data[18],"最大陣風風速時間(LST)":data[19],"降水量(mm)":data[20],"降水時數(hr)":data[21],"10分鐘最大降水量(mm)":data[22],"10分鐘最大降水起始時間(LST)":data[23]," 一小時最大降水量(mm)":data[24],"一小時最大降水量起始時間(LST)":data[25],"日照時數(hr)":data[26],"日照率(%)":data[27],"全天空日射量(MJ/㎡)":data[28],"能見度(km)":data[29],"A型蒸發量(mm)":data[30],"日最高紫外線指數":data[31],"日最高紫外線指數時間(LST)":data[32],"總雲量(0~10)":data[33],"year":data[34],"month":data[35],"day":data[36],"date":data[37]})
            day=day+1
            data=[]
        df=pd.concat([df,pd.DataFrame(fdata)], ignore_index=True,sort=True)
        df["date"] = pd.to_datetime(df["date"])
    path=r"C:\Users\admin\Desktop\Project\test"+"\\"+place_name[a]+".csv"
    df.to_csv(path, encoding='utf_8_sig')

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\admin\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
place=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北"]
for pl in place:
    weather=pd.read_csv(pl+".csv")
    weather=weather[['最低氣溫(℃)','最大陣風(m/s)','最大陣風風向(360degree)','最小相對溼度(%)','最高氣溫(℃)','氣溫(℃)','測站最低氣壓(hPa)','測站最高氣壓(hPa)','相對溼度(%)','風向(360degree)','測站氣壓(hPa)','降水量(mm)','風速(m/s)','year','month','day','date']]
    weather=weather.drop(weather.index[[-1]])
    column_list=list(weather.columns.values)
    column_list.pop(-1)
    for i in column_list:
        for j in range(weather.shape[0]):
            if str(weather[i][j]).replace(".", "", 1).isdigit()==False:
                if j-1<0:
                    weather[i][j]=0
                else:
                    weather[i][j]=weather[i][j-1]
    weather.to_csv("changed"+pl+".csv", encoding='utf_8_sig')  


C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
import pandas as pd
place_name=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北","新北","基隆","新竹","南投","宜蘭","花蓮","台東"]
for place in place_name:
    weather=pd.read_csv(r"C:\Users\admin\Desktop\Project\train_data"+"\\"+"天氣"+"\\"+place+".csv")
    weather=weather[['最低氣溫(℃)','最大陣風(m/s)','最大陣風風向(360degree)','最小相對溼度(%)','最高氣溫(℃)','氣溫(℃)','測站最低氣壓(hPa)','測站最高氣壓(hPa)','相對溼度(%)','風向(360degree)','測站氣壓(hPa)','降水量(mm)','風速(m/s)','year','month','day','date']]
    column_list=list(weather.columns.values)
    column_list.pop(-1)
    for i in column_list:
        for j in range(weather.shape[0]):
            if str(weather[i][j]).replace(".", "", 1).isdigit()==False:
                if j-1<0:
                    weather[i][j]=0
                else:
                    weather[i][j]=weather[i][j-1]
    weather.to_csv(r"C:\Users\admin\Desktop\Project\train_data"+"\\"+"天氣"+"\\"+place+".csv", encoding='utf_8_sig')  

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
place=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北"]
for i in place:
    crop=pd.read_csv("甘藍-初秋.csv")
    weather=pd.read_csv("changed"+i+".csv")
    crop=crop.drop(["Unnamed: 0"], axis=1)
    weather=weather.drop(["Unnamed: 0"], axis=1)
    train=weather.merge(crop, on='date', how='left')
    train=train.fillna(0)
    train=train.drop(["crop_name"], axis=1)
    train=train.drop(["crop_num"], axis=1)
    train=train.drop(["market_name"], axis=1)
    train=train.drop(["market_num"], axis=1)
    train.to_csv("train"+i+".csv", encoding='utf_8_sig')  

In [35]:
#web
place="台北一"
crop_num="LA1"
crop_name="甘藍-初秋"
col_name=["date","crop_num","crop_name","market_num","market_name","high","medium","low","mean","volume"] 
crop=pd.DataFrame(columns=col_name)
skip_num=0
temp_str=""
for j in range(0,1000): 
    url="https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?$top=1000&$skip="+str(skip_num)+"&Crop="+urllib.parse.quote(crop_name)+"&StartDate=101.01.01&EndDate=108.03.29&Market="+urllib.parse.quote(place) 
    get = json.loads(urlopen(url).read().decode('utf-8')) 
    data = [] 
    for i in range(0,len(get)):
        temp_str=get[0]['交易日期']
        Date=get[i]['交易日期'].split('.')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        get[i]['交易日期']=year+'/'+month+'/'+date
        if get[i]['作物代號']==crop_num[b]:
            data.append({"date":get[i]['交易日期'],"crop_num":get[i]['作物代號'],"crop_name":get[i]['作物名稱'],"market_num":get[i]['市場代號'],"market_name":get[i]['市場名稱'],"high":get[i]['上價'],"medium":get[i]['中價'],"low":get[i]['下價'],"mean":get[i]['平均價'],"volume":get[i]['交易量']}) 
        else:
            data.append({"date":get[i]['交易日期'],"crop_num":get[i]['作物代號'],"crop_name":get[i]['作物名稱'],"market_num":get[i]['市場代號'],"market_name":get[i]['市場名稱'],"high":0,"medium":0,"low":0,"mean":0,"volume":0}) 
    data.reverse() 
    crop=pd.concat([pd.DataFrame(data), crop], ignore_index=True,sort=True)
    if len(get)<1000: 
        break 
    skip_num=skip_num+1000 
    time.sleep(2000.0/1000.0)
crop["date"] = pd.to_datetime(crop["date"])

place_name="鞍部"
place_num="466910"
today_date="2019-04-06"
date=[]
for year in ['2012','2013','2014','2015','2016','2017','2018']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        date.append('-'.join([year,month]))
date.append('2019-01')
date.append('2019-02')
date.append('2019-03')
date.append('2019-04')
flag=0
name=urllib.parse.quote(urllib.parse.quote(place_name))
for dd in date:
    url="http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain"+"&station="+place_num+"&stname="+name+"&datepicker="+str(dd)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    trs = soup.findAll('tr')
    ths = trs[2].findAll('th')
    day=1
    data=[]
    fdata=[]
    if flag==0:
        title=[]
        for th in ths:
            title.append(th.text)
        title.pop(0)
        title.append("year")
        title.append("month")
        title.append("day")
        title.append("date")
        weather=pd.DataFrame(columns=title)
        flag=1
    for tr in trs[4:]:
        t=tr.findAll('td')
        for x in t:
            data.append(x.text.strip())
        sp=dd.split('-')
        data.pop(0)
        data.append(sp[0])
        data.append(sp[1])
        data.append(str(day))
        data.append(sp[0]+'/'+sp[1]+'/'+str(day))
        fdata.append({"測站氣壓(hPa)":data[0],"海平面氣壓(hPa)":data[1],"測站最高氣壓(hPa)":data[2],"測站最高氣壓時間(LST)":data[3],"測站最低氣壓(hPa)":data[4],"測站最低氣壓時間(LST)":data[5],"氣溫(℃)":data[6],"最高氣溫(℃)":data[7],"最高氣溫時間(LST)":data[8],"最低氣溫(℃)":data[9],"最低氣溫時間(LST)":data[10],"露點溫度(℃)":data[11],"相對溼度(%)":data[12],"最小相對溼度(%)":data[13],"最小相對溼度時間(LST)":data[14],"風速(m/s)":data[15],"風向(360degree)":data[16],"最大陣風(m/s)":data[17],"最大陣風風向(360degree)":data[18],"最大陣風風速時間(LST)":data[19],"降水量(mm)":data[20],"降水時數(hr)":data[21],"10分鐘最大降水量(mm)":data[22],"10分鐘最大降水起始時間(LST)":data[23]," 一小時最大降水量(mm)":data[24],"一小時最大降水量起始時間(LST)":data[25],"日照時數(hr)":data[26],"日照率(%)":data[27],"全天空日射量(MJ/㎡)":data[28],"能見度(km)":data[29],"A型蒸發量(mm)":data[30],"日最高紫外線指數":data[31],"日最高紫外線指數時間(LST)":data[32],"總雲量(0~10)":data[33],"year":data[34],"month":data[35],"day":data[36],"date":data[37]})
        day=day+1
        data=[]
    weather=pd.concat([weather,pd.DataFrame(fdata)], ignore_index=True,sort=True)
weather["date"] = pd.to_datetime(weather["date"])
weather=weather[['最低氣溫(℃)','最大陣風(m/s)','最大陣風風向(360degree)','最小相對溼度(%)','最高氣溫(℃)','氣溫(℃)','測站最低氣壓(hPa)','測站最高氣壓(hPa)','相對溼度(%)','風向(360degree)','測站氣壓(hPa)','降水量(mm)','風速(m/s)','year','month','day','date']]
a=weather[weather['date']==today_date].index.item()
weather=weather[:a+1]
column_list=list(weather.columns.values)
column_list.pop(-1)
for i in column_list:
    for j in range(weather.shape[0]):
        if str(weather[i][j]).replace(".", "", 1).isdigit()==False:
            if j-1<0:
                weather[i][j]=0
            else:
                weather[i][j]=weather[i][j-1]

                
train=weather.merge(crop, on='date', how='left')
train=train.fillna(0)
train=train.drop(["crop_name"], axis=1)
train=train.drop(["crop_num"], axis=1)
train=train.drop(["market_name"], axis=1)
train=train.drop(["market_num"], axis=1)
print(train)

In [5]:
crop=pd.read_csv("甘藍-初秋.csv")
a=crop[crop['date']=='2013-01-01'].index.item()
print(a)
b=crop.iloc[a:a+10]
print(b)

304
     Unnamed: 0 crop_name crop_num        date  high   low market_name  \
304         304     甘藍-初秋      LA1  2013-01-01  32.7  18.9         台北一   
305         305     甘藍-初秋      LA1  2013-01-02  28.5  17.8         台北一   
306         306     甘藍-初秋      LA1  2013-01-03  34.6  21.6         台北一   
307         307     甘藍-初秋      LA1  2013-01-04  33.4  22.3         台北一   
308         308     甘藍-初秋      LA1  2013-01-05  32.5  20.5         台北一   
309         309     甘藍-初秋      LA1  2013-01-06  30.7  19.8         台北一   
310         310     甘藍-初秋      LA1  2013-01-08  29.7  18.3         台北一   
311         311     甘藍-初秋      LA1  2013-01-09  24.2  13.3         台北一   
312         312     甘藍-初秋      LA1  2013-01-10  26.2  16.9         台北一   
313         313     甘藍-初秋      LA1  2013-01-11  27.5  18.6         台北一   

     market_num  mean  medium    volume  
304         109  25.7    25.6  131212.0  
305         109  23.3    23.5  144099.0  
306         109  28.1    28.1  107419.0  
307         1